In [26]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from datetime import datetime

In [27]:
df_listings = pd.read_csv('./listings.csv')
df_reviews = pd.read_csv('./reviews.csv')
df_listings = df_listings.convert_dtypes()
df_reviews = df_reviews.convert_dtypes()
df_listings = df_listings.rename(columns={'id':'listing_id'})


In [28]:
#to drop
cols_to_drop_listings = ['listing_url','scrape_id','last_scraped','source','picture_url','host_url','host_name','host_thumbnail_url',
                'host_picture_url','has_availability','license', 'availability_30','availability_60','availability_90','availability_365',
                'calendar_last_scraped','host_location','bathrooms_text','calendar_updated','host_id','host_neighbourhood']

cols_to_drop_reviews = ['reviewer_name','reviewer_id']

In [29]:
df_listings.drop(columns=cols_to_drop_listings,inplace=True)
df_reviews.drop(columns=cols_to_drop_reviews,inplace=True)
df_reviews.dropna(subset=['comments'],inplace=True)
df_reviews_combined = df_reviews.groupby('listing_id')['comments'].agg('``'.join).reset_index()
row_counts = df_reviews.groupby('listing_id').size().reset_index(name='review_count')
df_reviews = pd.merge(df_reviews_combined, row_counts, on='listing_id').reset_index()
df_combined = pd.merge(df_reviews,df_listings,on='listing_id').reset_index()
df_combined.drop(columns=['level_0','index'],inplace=True)
df_combined['host_response_rate'] = df_combined['host_response_rate'].str.rstrip('%').astype('float') / 100

In [30]:
def clean_string(text):
    if(type(text) == str):
        text = bs(text,'html.parser').get_text()
        tags = re.compile('<.*?>')
        text = re.sub(tags, '', text)
        rx = re.compile(r"[^A-Za-z0-9\s`]")
        text = rx.sub(' ', text)
        rx = re.compile(" +")
        text = rx.sub(' ', text).strip()
        if text == 't':
            return 1
        if text == 'f':
            return 0
        try:
            return (datetime.now() - datetime.strptime(text,"%Y %m %d")).days
        except:
            pass
        try:
            temp = text.split(' ')[0]
            pattern = r'^\d+$'
    
            if re.fullmatch(pattern,temp):
                return float(temp)
        except ValueError:
            pass
        if text == '':
            return None
    return text

In [31]:
df_combined = df_combined.applymap(clean_string).convert_dtypes()

/tmp/ipykernel_27811/436385161.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(clean_string).convert_dtypes()
/tmp/ipykernel_27811/3179302904.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = bs(text,'html.parser').get_text()
/tmp/ipykernel_27811/3179302904.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = bs(text,'html.parser').get_text()
/tmp/ipykernel_27811/3179302904.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = bs(text,'html.parser').get_text()
/tmp/ipykernel_27811/3179302904.py:3: MarkupResemblesLocatorWarning: The input looks more like a fi

In [33]:
df_combined.to_csv('combined.csv',index=False,encoding='utf-8')